In [1]:
from synthetic_data import *
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

from get_real_data import *

import tensorflow as tf
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import LSTM, Dense, Masking, Conv1D, Flatten, MaxPooling1D, Input, Concatenate, Dropout
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras import regularizers
import matplotlib.pyplot as plt


2023-09-02 12:19:09.228797: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-09-02 12:19:09.230156: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-02 12:19:09.260911: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-02 12:19:09.261954: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-02 12:19:09.780493: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not fin

In [2]:
X_ris_wedg, y_ris_wedg, X_fal_wedg, y_fal_wedg, X_d_top, y_d_top, X_d_bottom, y_d_bottom = get_X_y(noise=True, general=False)

In [3]:
X1, y1, X2, y2, X3, y3, X4, y4 = get_real_X_y(noise = True)

In [8]:
X = X_ris_wedg + X1
y = y_ris_wedg + y1

In [5]:
def onehotencode(y_synth):
    y_synth = np.array(y_synth)
    start = y_synth[:,0]
    end = y_synth[:,1]
    pattern = y_synth[:,2]
    mapping = {
        0: [0, 0, 0, 0],
        1: [0, 0, 0, 1],
        2: [0, 0, 1, 0],
        3: [0, 1, 0, 0],
        4: [1, 0, 0, 0]
    }
    mapped_values = [list(mapping[val]) for val in pattern]
    y_s = (np.column_stack((start, end, mapped_values)))
    return y_s

In [9]:
y_preprocessed = onehotencode(y)

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y_preprocessed, test_size=0.2)

In [11]:
y_train_dates = y_train[:,:2]
y_test_dates = y_test[:,:2]
y_train_p = y_train[:,2:]
y_test_p = y_test[:,2:]

In [12]:
X_train_processed=[]
scale={}
for i in range(len(X_train)):
    open_scaler_i = StandardScaler()
    X_train_processed.append(open_scaler_i.fit_transform(X_train[i]))
    scale[f"open_scaler_{len(X_train[i])}"] = open_scaler_i

X_test_processed=[]
scale={}
for i in range(len(X_test)):
    open_scaler_i = StandardScaler()
    X_test_processed.append(open_scaler_i.fit_transform(X_test[i]))
    scale[f"open_scaler_{len(X_test[i])}"] = open_scaler_i


In [20]:
X_train_preprocessed = pad_sequences(X_train_processed,maxlen = 502, dtype='float32', padding='post', value=-1)
X_test_preprocessed = pad_sequences(X_test_processed, maxlen = 502, dtype='float32', padding='post', value=-1)


In [21]:
X_train_preprocessed = tf.convert_to_tensor(X_train_preprocessed, np.float32)
y_train = tf.convert_to_tensor(y_train, np.int16)

display(type(X_train_preprocessed))
display(type(y_train))

tensorflow.python.framework.ops.EagerTensor

tensorflow.python.framework.ops.EagerTensor

In [22]:
def conditional_regression_model_1():
    # Input layer
    # Input layer
    input_shape = X_train_preprocessed.shape[1:]
    input_layer = Input(shape=input_shape)


    # Shared layers
    masked_input = Masking(mask_value=-1)(input_layer)
    conv1d_1 = Conv1D(32, activation='relu', kernel_size=3, kernel_regularizer=regularizers.L1L2(l1=1e-3, l2=1e-3))(masked_input)
    maxpool_1 = MaxPooling1D(pool_size=2)(conv1d_1)
    conv1d_2 = Conv1D(32, activation='relu', kernel_size=3)(maxpool_1)
    
    flatten = Flatten()(conv1d_2)
    dense_1 = Dense(units=192, activation='relu')(flatten)
    dropout_1 = Dropout(rate=0.5)(dense_1)
    dense_2 = Dense(units=128, activation='relu')(dropout_1)
    dropout_2 = Dropout(rate=0.5)(dense_2)
    dense_3 = Dense(units=64, activation='relu')(dropout_2)
    dropout_3 = Dropout(rate=0.5)(dense_3)
    dense_4 = Dense(units=32, activation='relu')(dropout_3)
    

    # Classification output
    classification_output = Dense(units=4, activation='softmax', name='classification_output')(dense_4)

    # Combine input and classification output for regression
    regression_input = Concatenate()([dense_4, classification_output])

    # Regression output
    regression_output = Dense(units=2, activation='linear', name='regression_output')(regression_input)

    # Create the combined model
    model = Model(inputs=input_layer, outputs=[classification_output, regression_output])

    # Compile the model with appropriate loss functions and metrics for each output
    model.compile(optimizer='adam',
                  loss={'classification_output': 'categorical_crossentropy', 'regression_output': 'mean_squared_error'},
                  metrics={'classification_output': 'accuracy', 'regression_output': 'mae'})

    return model


In [23]:
es = EarlyStopping(patience = 10, restore_best_weights=True)
#lr_scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, verbose=1, min_lr=1e-6)

model_3 = conditional_regression_model_1()

history = model_3.fit(
    x=X_train_preprocessed,
    y={'classification_output': y_train_p, 'regression_output': y_train_dates},
    validation_split = 0.2,
    shuffle = True,
    batch_size=32,
    epochs = 50,
    callbacks = [es],
    verbose = 1
)

Epoch 1/50
32/32 [==============================] - 1s 19ms/step - loss: 11745.4170 - classification_output_loss: 20.8179 - regression_output_loss: 11724.5410 - classification_output_accuracy: 0.0342 - regression_output_mae: 70.1412 - val_loss: 8405.0840 - val_classification_output_loss: 23.4905 - val_regression_output_loss: 8381.5352 - val_classification_output_accuracy: 0.0000e+00 - val_regression_output_mae: 61.9423
Epoch 2/50
32/32 [==============================] - 0s 15ms/step - loss: 8950.5566 - classification_output_loss: 23.8730 - regression_output_loss: 8926.6260 - classification_output_accuracy: 0.0372 - regression_output_mae: 65.9496 - val_loss: 7982.2329 - val_classification_output_loss: 15.8551 - val_regression_output_loss: 7966.3213 - val_classification_output_accuracy: 0.0000e+00 - val_regression_output_mae: 60.8304
Epoch 3/50
32/32 [==============================] - 0s 16ms/step - loss: 8123.4951 - classification_output_loss: 10.9001 - regression_output_loss: 8112.5371

In [24]:
evaluation_3 = model_3.evaluate(
    x=X_test_preprocessed,  # Test input data
    y={'classification_output': y_test_p, 'regression_output': y_test_dates},  # Dictionary of test labels
    verbose=1  # Set to 1 for progress updates during evaluation
)

10/10 [==============================] - 0s 6ms/step - loss: 3044.9692 - classification_output_loss: 0.0180 - regression_output_loss: 3044.8901 - classification_output_accuracy: 0.5897 - regression_output_mae: 34.9237
